<!-- #Consider .=0, a = 1, b = 2, c = 3, d = 4
Example - cab
 .cab.

 xs_c = [".","c","a","b"]
 ys_c = ["c","a","b","."]
 #we can also visualize it as
 ##[".c","ca","ab","b."]

 xs = [0,3,1,2]
 ys = [3,1,2,0]

 encode cab with 5 class

                .[1,0,0,0,0]
                c[0,0,0,1,0]
 xenc     =     a[0,1,0,0,0]     
                b[0,0,1,0,0]


     
                 [
                  [ 1.5674, -0.2373, -0.0274, -1.1008,  0.2859],
                  [-0.0296, -1.5471,  0.6049,  0.0791, -0.7814],
                  [-0.2808, -0.7389,  1.7249,  0.0380, -1.0694],
                  [-0.5374,  0.0511, -0.4755,  0.6205,  1.1500],
                  [-0.9643, -0.8815, -0.8295, -0.2548, -0.9648]
                 ]

logits_for_dot = [-0.2808, -0.7389,  1.7249,  0.0380, -1.0694] #3rd row -->

In [5]:

import torch


token = ".abcd"
stoi ={ char:index for index,char in enumerate(token)}
itos ={char:index  for index,char in stoi.items()}


data = "cab"
data_ind = ".cab."
xs,ys = [],[]

for x,y in zip(data_ind,data_ind[1:]):
    xs.append(stoi[x])
    ys.append(stoi[y])

xs =  torch.tensor(xs)
ys = torch.tensor(ys)
print(xs,ys)

tensor([0, 3, 1, 2]) tensor([3, 1, 2, 0])


In [6]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=5).float() # input to the network: one-hot encoding
print(xenc)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])


In [7]:
import torch

W = torch.tensor([

[ 0.2963, 1.1951, 0.7433, -0.1034, -0.6192],

[ 2.6439, 1.7843, 0.2086, 0.5000, 0.4406], 

[-0.6386, 0.2266, -0.0906, -0.3895, -0.2717],

[-0.2562, -0.7472, -0.8306, -1.2820, 0.2468],

[-1.3791, -1.8291, -0.4827, 0.2922, 0.4291]

], requires_grad=True)

print(W)

tensor([[ 0.2963,  1.1951,  0.7433, -0.1034, -0.6192],
        [ 2.6439,  1.7843,  0.2086,  0.5000,  0.4406],
        [-0.6386,  0.2266, -0.0906, -0.3895, -0.2717],
        [-0.2562, -0.7472, -0.8306, -1.2820,  0.2468],
        [-1.3791, -1.8291, -0.4827,  0.2922,  0.4291]], requires_grad=True)


In [8]:
#W = torch.randn((5, 5), requires_grad=True)
#print(W)
logits = xenc @ W
print(logits)

tensor([[ 0.2963,  1.1951,  0.7433, -0.1034, -0.6192],
        [-0.2562, -0.7472, -0.8306, -1.2820,  0.2468],
        [ 2.6439,  1.7843,  0.2086,  0.5000,  0.4406],
        [-0.6386,  0.2266, -0.0906, -0.3895, -0.2717]], grad_fn=<MmBackward0>)


In [9]:
import numpy as np
counts = logits.exp() # counts, equivalent to N

array = np.array(counts.data.tolist())
print(np.around(array, decimals=2))

probs = counts / counts.sum(1, keepdims=True)
array = np.array(probs.data.tolist())
print(np.around(array, decimals=2))

[[ 1.34  3.3   2.1   0.9   0.54]
 [ 0.77  0.47  0.44  0.28  1.28]
 [14.07  5.96  1.23  1.65  1.55]
 [ 0.53  1.25  0.91  0.68  0.76]]
[[0.16 0.4  0.26 0.11 0.07]
 [0.24 0.15 0.13 0.09 0.39]
 [0.58 0.24 0.05 0.07 0.06]
 [0.13 0.3  0.22 0.16 0.18]]


In [ ]:
-probs[torch.arange(len(xs)), ys].log().mean()

In [11]:
loss = -probs[torch.arange(len(xs)), ys].log().mean() + 0.05*(W**2).mean()
print(loss)
W.grad = None # set to zero the gradient
loss.backward()

tensor(2.3392, grad_fn=<AddBackward0>)


In [12]:
def cust_format(d):
    array = np.array(d.data.tolist())
    print(np.around(array, decimals=3))

In [13]:
cust_format(W.grad)


[[ 0.042  0.106  0.067 -0.223  0.014]
 [ 0.154  0.068 -0.237  0.019  0.018]
 [-0.221  0.077  0.055  0.039  0.045]
 [ 0.059 -0.216  0.03   0.016  0.1  ]
 [-0.006 -0.007 -0.002  0.001  0.002]]


In [14]:
W.data += -50 * W.grad
cust_format(W)


[[-1.815 -4.085 -2.614 11.041 -1.317]
 [-5.075 -1.616 12.037 -0.443 -0.442]
 [10.393 -3.61  -2.833 -2.359 -2.521]
 [-3.19  10.075 -2.345 -2.096 -4.739]
 [-1.103 -1.463 -0.386  0.234  0.343]]


In [10]:
aa = [ 0.2963, 1.1951, 0.7433, -0.1034, -0.6192]
aa = [ abs(num) for num in aa]
print(aa)
total = sum(aa)
print(total)
prob = [num/total for num in aa]
print(prob)

[0.2963, 1.1951, 0.7433, 0.1034, 0.6192]
2.9573
[0.10019274338078653, 0.40411862171575424, 0.25134413147127443, 0.03496432556724039, 0.20938017786494437]


In [17]:
out = []
ix = 0
while True:
 
    xenc = F.one_hot(torch.tensor([ix]), num_classes=5).float()
    logits = xenc @ W
        
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
        # ----------
        
    ix = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(itos[ix])
    if len(out) > 5:
        break
    if ix == 0:
        break
    print(''.join(out))

c
ca
cab
